In [ ]:
using CSV
using DataFrames
using JSON
using LargeScaleAnalysis
using Missings
using PyCall
using PyPlot

In [ ]:
DateFormatter = pyimport("matplotlib.dates").DateFormatter
DayLocator = pyimport("matplotlib.dates").DayLocator;

In [ ]:
using Dates
function PyPlot.plot(df, model; ax = gca())
    cmap = Dict(string(i) => plt.cm.tab20(i) for i in 1:model["n_states"])
    ax.plot(unix2datetime.(df.timestamp), coalesce.(df.PLT, NaN))
    for segment in model["segments"]
        span = unix2datetime.((segment["start_time"], segment["stop_time"]))
        ax.axvspan(span..., alpha = 0.3, color = cmap[segment["state"]])
    end
end

In [ ]:
df = CSV.read("/blobs/datasets/thesis/0001_0001_0.0.0.0_0.0.0.0.csv")
model = JSON.parsefile("/blobs/datasets/thesis/0001_0001_0.0.0.0_0.0.0.0.csv.json");

In [ ]:
@pywith plt.style.context("classicthesis") begin
    fig, ax = subplots(figsize = (4.6, 0.4*4.6))
    plot(df, model, ax = ax)
    ax.set(ylabel = "Page Load Time (s)", title = "baidu.com\nFibre Orange, Chrome v63, HTTP/2")
    ax.xaxis.set_major_formatter(DateFormatter("%b\n%Y"))
    ax.xaxis.set_major_locator(DayLocator(15))
    fig.savefig("../figures/webview-baidu.pdf")
end